In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp django.views

<IPython.core.display.Javascript object>

In [ ]:
# export
from pathlib import Path

from django.http import JsonResponse
from django.http import FileResponse

<IPython.core.display.Javascript object>

## Use Own FileResponse to Measure Serving Time

In [ ]:
# export

import time

from django.core import signals


class MeasuringFileResponse(FileResponse):
    def __init__(self, *args, **kwargs):
        # open_file = args[0]
        # print(f"starting to serve: {open_file.name}")
        self.open_file_name = args[0].name
        self.started_serving = time.perf_counter()
        super().__init__(*args, **kwargs)

    def close(self):
        elapsed = time.perf_counter() - self.started_serving
        print(f"elapsed: {elapsed} for {self.open_file_name}")
        for closer in self._resource_closers:
            try:
                closer()
            except Exception:
                pass
        # Free resources that were still referenced.
        self._resource_closers.clear()
        self.closed = True
        signals.request_finished.send(sender=self._handler_class)

<IPython.core.display.Javascript object>

## Django Views

In [ ]:
# export


async def serve_file_sync(request, base, path, num):
    file_path = Path(base) / path / str(num)
    # return JsonResponse({"file_path": file_path})
    return MeasuringFileResponse(file_path.open("rb"))
    # return FileResponse(file_path.open("rb"))

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

import aiofiles


class StubIterator:
    chunk_size = 4096

    def __init__(self, file):
        self.file = file
        self.more_body = True

    def __iter__(self):
        return self

    def __next__(self):
        chunk = await self.file.read(self.chunk_size)
        self.more_body = len(chunk) == self.chunk_size
        return chunk
        while self.more_body:
            chunk = await self.file.read(self.chunk_size)
            self.more_body = len(chunk) == self.chunk_size
            return chunk


class StubResponse:
    def __init__(self):
        async with aiofiles.open(self.path, mode="rb") as file:
            return StubIterator(file)

SyntaxError: 'await' outside async function (<ipython-input-30-30a1936a8664>, line 13)

<IPython.core.display.Javascript object>

In [ ]:
# export


class AsyncFileResponse(FileResponse):
    async_file_response = True
    chunk_size = 4096



async def serve_file(request, base, path, num):
    file_path = Path(base) / path / str(num)
    # print(f"serving file {file_path}")
    response = AsyncFileResponse(filename=file_path)
    response.headers["Content-Type"] = "application/octet-stream"
    #response = FileResponse(file_path.open("rb"))
    #response.async_file_response = False
    return response
    # return AsyncFileResponse(file_path)

<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_from_config.ipynb.
Converted 52_generate_files_for_benchmark.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.


<IPython.core.display.Javascript object>